# Physical Distance Maintaining

<font size ="3"><b>Work Flow:</b>
    
   <b> [Object detection] -->  [seprating person from other objects] --> [calculating distance b/w person] --> [Predicting Output]</b></font>
   
   
<b><font size = "3">Additional Files Required :

1) yolv3.config   
2) yolov3.weights   
3) coco.names</font></b>   
   

<font size = "4"><b><u>Importing Libraries</u></b></font>

In [1]:
from scipy.spatial import distance as dist #used for calculating eucledian distance b/w two persons

In [2]:
import numpy as np #used for using and manipulating array

In [3]:
import imutils #used for manipulating here especially used for resize the frame for easy detection

In [4]:
import cv2 #used for capturing and displaying the live feed and various other functionality

In [5]:
import os #used for manipulating paths in our system

<font size = "4"><b><u>Declaring and initiliazing variable</u></b></font>

In [6]:
location = "C:/Users/VIVEK/Desktop/social distance maintaing using eucledian" #path to our current folder

In [7]:
MIN_DISTANCE = 50 #minimum distance b/w two people considered to be safe

In [8]:
MIN_CONF = 0.3 #minimum confidence to filter out the bounding box 

In [9]:
NMS_THRESH = 0.3 #minimum threshold used in Non maximum supression for filtering out overlapping boxes

In [10]:
path = os.path.sep.join([location,"coco.names"]) #path of file containing names of object that YOLOv3 can recognize

In [11]:
labels = open(path).read().strip().split("\n") #storing all the coco names in an array 

In [12]:
weight_path = "C:/Users/VIVEK/Desktop/social distance maintaing using eucledian/yolov3.weights" # path of file containing weights for objects

In [13]:
config_path = "C:/Users/VIVEK/Desktop/social distance maintaing using eucledian/yolov3.cfg" #path of the file containing configuration of yolov3

<font size = "4"><b><u>Loading the Yolo Object detector</u></b></font>

In [14]:
net = cv2.dnn.readNetFromDarknet(config_path,weight_path) #used to load Yolo object detector network model

In [15]:
ln = net.getLayerNames() #returns list of all the layer contained in our network defined by net

print(len(ln))

ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()] #it gives you the names of final output layers

print(ln)

254
['yolo_82', 'yolo_94', 'yolo_106']


<font size = "4"><b><u>specifying input (either live feed or pre recorded video)</u></b></font>

In [16]:
vs = cv2.VideoCapture("demoSD2.mp4") #opening a pre recorded video

# vs = cv2.VideoCapture(0) #use when you want to access the live feed

writer = None #variable for storing whether we have a frame to read or not 

<font size = "4"><b><u>Function to detect people and getting there cordinates</u></b></font>

In [17]:
def detect_people(frame, net, ln, personIdx=0):
    
    (H, W) = frame.shape[:2] # grabing the height and width of the frame
    
    results = []

    #constructing a blob from input frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    
    #specifying to network to consider blob as its input
    net.setInput(blob)
    
    #forward passing the output layers to obtain all the bounding boxes 
    layerOutputs = net.forward(ln)

    boxes = [] # list for stroing the bounding boxes
    centroids = [] #list for storing the centroid of each box
    confidences = [] # list of confidence for each box

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter detections by ensuring that the object
            # detected was a person and that the minimum
            # confidence is met
            if classID == personIdx and confidence > MIN_CONF:
                #since we have converted our input frame into a blob of size 416X416 therefore we have to trace back
                #the real co-ordinates for bounding box relative to input size
                box = detection[0:4] * np.array([W, H, W, H])
                
                #storing center and height and width from box into variable as int type
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box so that it can be use to draw boxes in output file
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                #inserting respective info in boxes, centroids and confidences
                
                boxes.append([x, y, int(width), int(height)])  #co-ordinates of a boxes
                centroids.append((centerX, centerY)) # center point of a bounding box
                confidences.append(float(confidence)) #confidence of a particular box

    # apply non-maxima suppression to suppress weak, overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # update our results list to consist of the person
            # prediction probability, bounding box coordinates,
            # and the centroid
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    # return the list of results
    return results

<font size="3"><b><u>Stating the name and format for out output file</u></b></font>

In [18]:
display = 1
output = "Prediction.avi"

<font size="3"><b><u>Accessing the video frame by frame and predicting the output</u></b></font>

In [19]:
while True:
    # grabbing the video frame by frame
    (grabbed, frame) = vs.read() #grabbed will contain whether a frame is detected or not i.e a boolean value and
    # frame will store 

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        cv2.destroyAllWindows()
        break

    # resize the frame and then detect people (and only people) in it
    frame = imutils.resize(frame, width=700)
    results = detect_people(frame, net, ln, personIdx=labels.index("person"))

    
    violate = set() # contains the co-ordinates of peoples voilating social distancing 

    #if there is only one person then there is no need to check social distancing 
    if len(results) >= 2:
        #storing all the co-ordintes of bounding boxes for calculating the eucledian distance
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")

        # loop over the upper triangular of the distance matrix
        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                # check to see if the distance between any two
                # centroid pairs is less than the configured number
                # of pixels
                if D[i, j] < MIN_DISTANCE:
                    # update our violation set with the indexes of the centroid pairs
                    violate.add(i)
                    violate.add(j)

    # loop over the results
    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid
        color = (0, 255, 0)

        # if the index pair exists within the violation set, then update the color
        if i in violate:
            color = (0, 0, 255)

        # draw a bounding box around the person
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # if an output video file path has been supplied and the video
    # writer has not been initialized, do so now
    if output != "" and writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output, fourcc, 25, (frame.shape[1], frame.shape[0]), True)

    # if the video writer is not None, write the frame to the output video file
    if writer is not None:
        writer.write(frame)
        
    if display > 0:
        #show the output frame
        cv2.imshow("Output window", frame)
        key = (cv2.waitKey(1)&0xFF)
        if key == ord('x'):
            vs.release()
            cv2.destroyAllWindows()
            break

<font size ="4"><b>Thank you! </b></font>